In [3]:
# Install dependencies
%pip install -U --force-reinstall llama-index llama-index-embeddings-huggingface llama-index-llms-llama-cpp guidance transformers[torch] ragas datasets pandas langchain-core langchain-community sentence-transformers
# %pip install llama-cpp-python # without cuda support
%PATH="/usr/local/cuda-12.3/bin:$PATH" CMAKE_ARGS="-DLLAMA_CUBLAS=on" pip install --force-reinstall --no-cache llama-cpp-python # For CUDA support

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)



[notice] A new release of pip is available: 23.2.1 -> 24.0
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


UsageError: Line magic function `%PATH="/usr/local/cuda-12.3/bin:$PATH"` not found.


In [1]:
# Our basic llama-index RAG setup
from llama_index.core import (
    SimpleDirectoryReader,
    VectorStoreIndex,
    # set_global_handler
)
from llama_index.embeddings.huggingface import HuggingFaceEmbedding
from llama_index.llms.llama_cpp import LlamaCPP
from llama_index.llms.llama_cpp.llama_utils import (
    messages_to_prompt,
    completion_to_prompt,
)

# Debugging for checking Context and LLM Output
# set_global_handler("simple")

# Initialize the LLM (e.g., LlamaCPP with your model)
llm = LlamaCPP(
    model_path="./mistral-7b-instruct-v0.2.Q4_K_M.gguf",
    temperature=0.0,
    max_new_tokens=2048,
    context_window=4096,
    generate_kwargs={},
    model_kwargs={"n_gpu_layers": -1},
    messages_to_prompt=messages_to_prompt,
    completion_to_prompt=completion_to_prompt,
    verbose=False
)

# Initialize the embedding model
embed_model = HuggingFaceEmbedding(model_name="BAAI/bge-small-en-v1.5")



# Load documents
documents = SimpleDirectoryReader("./data", recursive=True).load_data()

# Create an index from the documents using the service context
index = VectorStoreIndex.from_documents(documents, embed_model=embed_model)

# Now you can use the index to perform queries
query_engine = index.as_query_engine(llm=llm, similarity_top_k=5,  chunk_size=256, chunk_overlap=64)

/home/daniel/code/llamaindex-rag-glossary/.venv/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
ggml_init_cublas: GGML_CUDA_FORCE_MMQ:   no
ggml_init_cublas: CUDA_USE_TENSOR_CORES: yes
ggml_init_cublas: found 1 CUDA devices:
  Device 0: NVIDIA GeForce RTX 3060, compute capability 8.6, VMM: yes


In [2]:
# Testing the RAG Setup with an example question
response = query_engine.query("What is the similarity map?")
print(response)

 The Similarity Map is an interactive visualization component in Renumics Spotlight that projects datapoints onto a 2D map based on the similarity of selected columns from datasets loaded using the Hugging Face library. The similarity can be measured by pre-computed descriptions like embeddings or by scalar columns in the dataset. Users can select columns to 'placeBy' in the settings and watch the dimension reduction arrange the data on the map using UMAP or PCA algorithms. The map can be moved, zoomed, and selected points can be added or removed using mouse controls. Coloring and sizing can also be used to represent additional columns on the map. Renumics Spotlight directly works on top of the Hugging Face datasets library, allowing for efficient data visualization and inspection without the need for data copying or pre-processing.


In [3]:
# Evaluating our current pipeline and identifying shortcomings.
# Evaluating our current pipeline and identifying shortcomings.
EXAMPLE_QUESTIONS_WITH_GT = [
    {"question": "What is the similarity map?",
     "answer": "A similarity map is a visual representation or a data structure that shows how different pieces of data are similar to each other."
    },
    {
        "question": "How can I install Renumics Spotlight?",
        "answer": "You can install Renumics Spotlight by downloading it from the official website and following the installation instructions."
    },
    {
        "question": "I want to find duplicates in my data. How can I do this?",
        "answer": "To find duplicates in your data, you can use data deduplication tools or software that scans your dataset for duplicate entries."
    },
    {
        "question": "I want to look at my audio data. Which possibilities does Spotlight offer?",
        "answer": "Spotlight offers features like audio visualization, audio data analysis, and the ability to search through audio files using metadata or content-based queries."
    },
]

In [4]:
import pandas as pd

# Initialize a list to hold your data
data = []

# Iterate over each question
for item in EXAMPLE_QUESTIONS_WITH_GT:
    question = item["question"]

    ground_truth = item["answer"] # Those could be multiple
    
    # Query your RAG system
    response = query_engine.query(question)
    
    # Assuming the response object has the answer
    answer = response.response  # Modify as per your response object structure
    
    # Extract context from the source_nodes attribute
    # Concatenate the text of each source node to form the full context
    contexts = [node.text for node in response.source_nodes]
    
    # Append the details to your data list
    data.append({"question": question, "ground_truth": ground_truth, "answer": answer, "contexts": contexts})

# Convert the list to a pandas DataFrame
df = pd.DataFrame(data)

In [5]:
df

,question,ground_truth,answer,contexts
0,What is the similarity map?,A similarity map is a visual representation or...,The Similarity Map is an interactive visualiz...,[---\ntags: []\nid: similarity-map\nsidebar_po...
1,How can I install Renumics Spotlight?,You can install Renumics Spotlight by download...,"To install Renumics Spotlight, you can use pi...",[---\nslug: /docs/development\ntitle: Contribu...
2,I want to find duplicates in my data. How can ...,"To find duplicates in your data, you can use d...","To find duplicates in your data, you can use ...",[---\ntags: []\nid: duplicates-annoy\nsidebar_...
3,I want to look at my audio data. Which possibi...,Spotlight offers features like audio visualiza...,"Based on the context information provided, Sp...","[`audio(column, window_column=None, name=None,..."


In [6]:
df.to_json("rag_results.json")

